In [1]:
from fastai.vision.all import *
import fastai
from wwf.vision.timm import *
import timm

In [2]:
!nvidia-smi

Fri Mar 26 09:54:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:19:00.0 Off |                  N/A |
| 27%   37C    P0    65W / 250W |     10MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 30%   39C    P0    50W / 250W |     10MiB / 10989MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
import torch
torch.cuda.set_device(2)

In [4]:
path = Path('datasetCorregido2/')
trainPath = path/'train'
testPath = path/'test'

In [5]:
dbTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=GrandparentSplitter(valid_name='test'),
                 get_y=parent_label,
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=224, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])
dlsTest = dbTest.dataloaders(path,bs=32)

In [6]:
learn = Learner(dlsTest, timm.models.hrnet.hrnet_w32(num_classes=2),metrics=[accuracy,Precision(),Recall()],loss_func=LabelSmoothingCrossEntropy()).to_fp16()

In [7]:
learn.load('hrnetw32')

FileNotFoundError: [Errno 2] No such file or directory: 'models/hrnetw32.pth'

In [8]:
yPw40,y=learn.get_preds()

In [7]:
def getDataBlock2(presize=512,size=384):
    return DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=GrandparentSplitter(valid_name='test'),
                 get_y=parent_label,
                 item_tfms = [Resize(presize)], # CropPad(200,200)
                 batch_tfms=[*aug_transforms(size=size, min_scale=0.75,do_flip=True,flip_vert=True,
                      max_rotate=180,max_warp=0,max_zoom=1.1), Normalize.from_stats(*imagenet_stats)])

db = getDataBlock2()
dlsTest2 = db.dataloaders(path,bs=32)

In [25]:
from fastai.vision.all import *
callbacks = [
    ShowGraphCallback(),
    EarlyStoppingCallback(patience=5),
    SaveModelCallback(fname='vit'),
    ReduceLROnPlateau(patience=1)
]

In [26]:
import timm

In [27]:
learn = Learner(dlsTest2,timm.models.vision_transformer.vit_base_resnet50_384(num_classes=2,pretrained=True),metrics=[accuracy,Precision(),Recall(),RocAucBinary()],loss_func=LabelSmoothingCrossEntropy()).to_fp16()

In [28]:
learn.load('vit')

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [29]:
yPvit,y=learn.tta()

In [34]:
learn = Learner(dlsTest2,timm.models.vision_transformer.vit_deit_base_patch16_384(num_classes=2,pretrained=True),metrics=[accuracy,Precision(),Recall(),RocAucBinary()],loss_func=LabelSmoothingCrossEntropy()).to_fp16()

In [35]:
learn.load('deit')

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [36]:
yPdeit,y=learn.get_preds()

In [37]:
learn.metrics=[accuracy,Precision(),Recall(),RocAucBinary()]

In [38]:
learn.validate()

(#5) [0.49006423354148865,0.7942999005317688,0.8356481481481481,0.7916666666666666,0.883531389013845]

In [42]:
preds,y=learn.tta(n=6)

In [43]:
predIdxs = np.argmax(yPdeit.numpy(), axis=1)

In [44]:
tn, fp, fn, tp = confusion_matrix(predIdxs, y).ravel()

print(tn / (tn+fp))

0.7466666666666667


In [29]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,roc_auc_score

In [30]:
roc_auc_score(y,preds[:,1])

0.8996601189583646

In [18]:
from fastai.vision.all import *
callbacks = [
    ShowGraphCallback(),
    EarlyStoppingCallback(patience=3),
    SaveModelCallback(fname='efficientnet_b1'),
    MixUp(),
]

In [19]:
learn = timm_learner(dlsTest, 'efficientnet_b1',cbs = callbacks,metrics=[accuracy,Precision(),Recall()]).to_fp16()

In [20]:
learn.load('efficientnet_b1')

In [21]:
yPEffi,y=learn.get_preds()

In [22]:
from fastai.vision.all import *
callbacks = [
    ShowGraphCallback(),
    EarlyStoppingCallback(patience=3),
    SaveModelCallback(fname='resnet101'),
    MixUp(),
]

In [23]:
learn = timm_learner(dlsTest, 'resnet101',cbs = callbacks,metrics=[accuracy,Precision(),Recall()]).to_fp16()

No pretrained weights exist for this model. Using random initialization.


In [24]:
learn.load('resnet101')

In [25]:
yPRes,y=learn.get_preds()

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
preds=(yPvit+yPw40+yPdeit+yPEffi+yPRes)/5

In [27]:
predIdxs = np.argmax(preds.numpy(), axis=1)

In [28]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

In [29]:
precision_score(predIdxs,y)

0.7719298245614035

In [30]:
recall_score(predIdxs,y)

0.8523002421307506

In [31]:
f1_score(predIdxs,y)

0.810126582278481

In [32]:
accuracy_score(predIdxs,y)

0.7955390334572491

-------------------------------------------------------